# 電影推薦器系統 Movie Recommendation

In [2]:
# Re-importing necessary libraries and reloading the CSV to continue from where the session was interrupted
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the CSV file again
df = pd.read_csv('/kaggle/input/netflix-data-analysis/netflix_movies (1).csv')

array(['Find Yourself', 'Melodies of Life - Born This Way',
       'Dancing Angels', 'Comedy Premium League', 'Yeh Meri Family'],
      dtype=object)

## 1.填補數據集中缺失的值：

使用 fillna('') 方法來替換資料中的 NaN 值，將其轉為空字串，這樣可以避免處理文本時出錯 

In [3]:
# Data Preprocessing: Fill missing valuesb
df['director'] = df['director'].fillna('')
df['cast'] = df['cast'].fillna('')
df['country'] = df['country'].fillna('')
df['rating'] = df['rating'].fillna('')
df['listed_in'] = df['listed_in'].fillna('')


將多個特徵（如導演、演員、類型、分級等）結合成一個新的欄位 combined_features。這樣能夠將每個電影或節目所有相關的資訊合併，方便後續進行相似度計算。

In [5]:
# Combining features into a single column
df['combined_features'] = df['director'] + ' ' + df['cast'] + ' ' + df['listed_in'] + ' ' + df['rating']

## 2. 文本向量化：

使用 TfidfVectorizer，這是一種常用的自然語言處理技術。它根據每個詞在文檔中的頻率和整個資料集中出現的頻率來賦予不同的權重。這有助於去除過於常見的詞，讓模型更加關注具有區別性的重要詞。
tfidf.fit_transform(df['combined_features']) 將這些組合後的特徵轉換成數值化的向量，這些向量用來表示每部電影的文本特徵。

In [6]:
# Vectorizing the combined features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

## 3. 計算餘弦相似度：

cosine_similarity(tfidf_matrix, tfidf_matrix) 計算每兩部電影或節目之間的相似度。餘弦相似度的範圍是0到1，值越接近1表示兩個項目越相似。

In [7]:
# Computing cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

## 4. 推薦系統函數：

函數 get_recommendations(title) 接受一個電影或節目的標題作為輸入。
idx = df[df['title'].str.contains(title, case=False, na=False)].index[0]：這行程式碼會根據使用者輸入的標題找到對應電影或節目的索引。
計算這部電影與其他所有電影的相似度，並將結果按相似度從高到低排序。
sim_scores = sim_scores[1:6]：這裡選取相似度最高的前五個電影（忽略第一個因為它是自己）。

In [8]:
# Function to get recommendations based on a given title
def get_recommendations(title, cosine_sim=cosine_sim):
    # Find the index of the movie based on its title
    idx = df[df['title'].str.contains(title, case=False, na=False)].index[0]
    
    # Calculate similarity scores with other movies
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top 5 recommendations
    sim_scores = sim_scores[1:6]
    
    # Retrieve the corresponding titles
    movie_indices = [i[0] for i in sim_scores]
    recommended_titles = df['title'].iloc[movie_indices].values
    
    return recommended_titles

## 5. 使用者電影推薦器

最後，函數返回相似的電影或節目名稱。

In [10]:
# Testing the recommendation system with a sample movie title
sample_recommendations = get_recommendations('Kota Factory')
sample_recommendations

array(['Find Yourself', 'Melodies of Life - Born This Way',
       'Dancing Angels', 'Comedy Premium League', 'Yeh Meri Family'],
      dtype=object)